# Making Python Faster

## Standard python code

In [1]:
def fib(n):
    if n<2:
        return n
    return fib(n-1)+fib(n-2)

print(fib(30))

832040


In [2]:
%timeit fib(30)

330 ms ± 8.62 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Using Cython

In [3]:
%load_ext Cython

In [4]:
%%cython

def fib_cython(n):
    if n<2:
        return n
    return fib_cython(n-1) + fib_cython(n-2)

print(fib_cython(30))

832040


In [5]:
%timeit fib_cython(30)

74.4 ms ± 1.83 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Using Cython and static typing

In [6]:
%%cython
 
cpdef long fib_cython_type(long n):
    if n<2:
        return n
    return fib_cython_type(n-1)+fib_cython_type(n-2)

print(fib_cython_type(30))

832040


In [7]:
%timeit fib_cython_type(30)

3.31 ms ± 49.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Caching Computation

In [8]:
from functools import lru_cache as cache

@cache(maxsize=None)
def fib_cache(n):
    if n<2:
        return n
    return fib_cache(n-1)+fib_cache(n-2)

print(fib_cache(30))

832040


In [9]:
%timeit fib_cache(30)

97.9 ns ± 0.823 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


## Iteration

In [10]:
def fib_seq(n):
    if n < 2:
        return n
    a,b = 1,0
    for i in range(n-1):
        a,b = a+b,a
    return a   

print(fib_seq(30))

832040


In [11]:
%timeit fib_seq(30)

1.62 µs ± 23.3 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [12]:
%%cython

def fib_seq_cython(n):
    if n < 2:
        return n
    a,b = 1,0
    for i in range(n-1):
        a,b = a+b,a
    return a    

cpdef long fib_seq_cython_type(long n):
    if n < 2:
        return n
    cdef long a,b
    a,b = 1,0
    for i in range(n-1):
        a,b = a+b,a
    return a

print(fib_seq_cython(30))
print(fib_seq_cython_type(30))

832040
832040


In [13]:
%timeit fib_seq_cython(30)
%timeit fib_seq_cython_type(30)

875 ns ± 9.76 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
62.6 ns ± 0.8 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


## Compiling With Numba
Let us use another tool called Numba.  It is a just in time (jit) compiler for a subset of Python.  It does not work yet on all of Python, but when it does work it can do marvels. 

In [14]:
from numba import jit

In [15]:
@jit
def fib_seq_numba(n):
    if n < 2:
        return n
    a,b = 1,0
    for i in range(n-1):
        a,b = a+b,a
    return a   

print(fib_seq_numba(30))

832040


In [16]:
%timeit fib_seq_numba(30)

163 ns ± 0.999 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


## Using Numpy
Let's now look at the second benchmark.  It is an implementation of the quicksort algorithm.

In [17]:
def qsort_kernel(a, lo, hi):
    i = lo
    j = hi
    while i < hi:
        pivot = a[(lo+hi) // 2]
        while i <= j:
            while a[i] < pivot:
                i += 1
            while a[j] > pivot:
                j -= 1
            if i <= j:
                a[i], a[j] = a[j], a[i]
                i += 1
                j -= 1
        if lo < j:
            qsort_kernel(a, lo, j)
        lo = i
        j = hi
    return a

In [18]:
import random
 
def benchmark_qsort():
    lst = [ random.random() for i in range(1,5000) ]
    qsort_kernel(lst, 0, len(lst)-1)

In [19]:
%timeit benchmark_qsort

19.2 ns ± 0.141 ns per loop (mean ± std. dev. of 7 runs, 100000000 loops each)


In [20]:
%%cython
import numpy as np
import cython

@cython.boundscheck(False)
@cython.wraparound(False)
cdef double[:] \
qsort_kernel_cython_numpy_type(double[:] a, \
                               long lo, \
                               long hi):
    cdef: 
        long i, j
        double pivot
    i = lo
    j = hi
    while i < hi:
        pivot = a[(lo+hi) // 2]
        while i <= j:
            while a[i] < pivot:
                i += 1
            while a[j] > pivot:
                j -= 1
            if i <= j:
                a[i], a[j] = a[j], a[i]
                i += 1
                j -= 1
        if lo < j:
            qsort_kernel_cython_numpy_type(a, lo, j)
        lo = i
        j = hi
    return a

def benchmark_qsort_numpy_cython():
    lst = np.random.rand(5000)
    qsort_kernel_cython_numpy_type(lst, 0, len(lst)-1)

In [21]:
%timeit benchmark_qsort_numpy_cython

19.3 ns ± 0.374 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [22]:
def benchmark_sort_numpy():
    lst = np.random.rand(5000)
    np.sort(lst)

%timeit benchmark_sort_numpy

22.3 ns ± 0.402 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


## Profiling Code
Let use now look at a third example, computing the Mandelbrot set.  Julia team used this Python code:

In [23]:
def mandel(z):
    maxiter = 80
    c = z
    for n in range(maxiter):
        if abs(z) > 2:
            return n
        z = z*z + c
    return maxiter

def mandelperf():
    r1 = np.linspace(-2.0, 0.5, 26)
    r2 = np.linspace(-1.0, 1.0, 21)
    return [mandel(complex(r, i)) for r in r1 for i in r2]

%timeit mandelperf

21.1 ns ± 0.401 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [24]:
@jit
def mandel_numba(z):
    maxiter = 80
    c = z
    for n in range(maxiter):
        if abs(z) > 2:
            return n
        z = z*z + c
    return maxiter

def mandelperf_numba():
    r1 = np.linspace(-2.0, 0.5, 26)
    r2 = np.linspace(-1.0, 1.0, 21)
    c3 = [complex(r,i) for r in r1 for i in r2]
    return [mandel_numba(c) for c in c3]

%timeit mandelperf_numba

21.8 ns ± 0.88 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


# Line_Profiler
```
pip install line_profiler
```

In [25]:
%load_ext line_profiler

In [26]:
%lprun -s -f mandelperf_numba mandelperf_numba()

In [27]:
@jit
def mandelperf_numba_mesh():
    width = 26
    height = 21
    r1 = np.linspace(-2.0, 0.5, width)
    r2 = np.linspace(-1.0, 1.0, height)
    mandel_set = np.empty((width,height), dtype=int)
    for i in range(width):
        for j in range(height):
            mandel_set[i,j] = mandel_numba(r1[i] + 1j*r2[j])
    return mandel_set

In [28]:
%timeit mandelperf_numba_mesh

21.4 ns ± 0.831 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)
